In [ ]:
import pandas as pd

data = pd.read_excel('Raw_Data.xlsx', sheet_name = 'Data')

In [ ]:
#choose years to remove
start_year = 1995
end_year = 1999

omit_years = list(range(start_year,end_year,1))
for year in omit_years:
    data = data[data.year != year]

In [ ]:
#PB ratio strategy

#list of dataframes to output to excel
final_dataframes = []

#group data by year
grouped = data.groupby(['year'])

#set up dictionaries to contain highest and lowest PBs
lowest_PB_returns = {}
highest_PB_returns = {}

for name, group in grouped:
    #calculate percentiles of each PB relative to its group
    group['PB_percentile'] = group['PB'].rank(pct = True)

    #find bottom 10 percentile PB firms
    lowest_PB = group.loc[group['PB_percentile'] < 0.1]

    #calculate and store mean return of lowest PB in dictionary with the date as key
    lowest_PB_returns[name]=lowest_PB['ret'].mean()

    #find top 10 percentile PB firms
    highest_PB = group.loc[group['PB_percentile'] > 0.9]

    #calculate and store mean return of lowest PB in dictionary with the date as key
    highest_PB_returns[name]=highest_PB['ret'].mean()

#convert dictionaries to dataframes
dfl = pd.DataFrame.from_dict(lowest_PB_returns, orient = 'index', columns= ['avg_ret_LPB'])    
dfh = pd.DataFrame.from_dict(highest_PB_returns, orient = 'index', columns = ['avg_ret_HPB'])

#merge dataframes
df_pb = pd.concat([dfl,dfh], axis = 1)

#calculate hedged portfolio return as average return of bottom 10 percentile PB minus top 10 percentile PB
df_pb['avg_ret_hedged'] = df_pb['avg_ret_LPB']-df_pb['avg_ret_HPB']

#calculate average return and its standard deviation
df_pb_mean = df_pb.mean().to_frame(name = 'Average').T
df_pb_std = df_pb.std().to_frame(name = 'STD').T

#merge dataframes
df_pb = pd.concat([df_pb,df_pb_mean,df_pb_std])

#add dataframe to list exported to excel
final_dataframes.append(df_pb)


In [ ]:
#Accruals Strategy

#calculate accruals
data['accruals'] = (data['ni']-data['oancf'])/data['at']

grouped = data.groupby(['year'])

#dictionaries for the accruals

lowest_accruals = {}
highest_accruals = {}

for name, group in grouped:
    group['acrl_percentiles'] = group['accruals'].rank(pct = True)

    la = group.loc[group['acrl_percentiles']<0.1]

    lowest_accruals[name] = la['ret'].mean()

    ha = group.loc[group['acrl_percentiles']>0.9]

    highest_accruals[name] = ha['ret'].mean()

dfla = pd.DataFrame.from_dict(lowest_accruals, orient = 'index', columns = ['lowest accrual returns'])
dfha = pd.DataFrame.from_dict(highest_accruals, orient = 'index', columns =['highest accrual returns'])

dfa = pd.concat([dfla,dfha],axis = 1)

#calculate hedged portfolio returns
dfa['hedged_returns'] = dfha['highest accrual returns'] - dfla['lowest accrual returns']

dfa_mean = dfa.mean().to_frame(name = 'Average').T
dfa_std = dfa.std().to_frame(name = 'STD').T

dfa = pd.concat([dfa,dfa_mean,dfa_std])

final_dataframes.append(dfa)

In [ ]:
#Capex to revenue signal

data['capx_to_revenue'] = data['capx']/data['revt']

grouped = data.groupby(['year'])

low_ctr = {}
high_ctr = {}

for name, group, in grouped:
    group['ctr_percentiles'] = group['capx_to_revenue'].rank(pct = True)

    lowest_ctr = group.loc[group['ctr_percentiles'] < 0.1]

    low_ctr[name] = lowest_ctr['ret'].mean()

    highest_ctr = group.loc[group['ctr_percentiles'] > 0.9]

    high_ctr[name] = highest_ctr['ret'].mean()


dflc = pd.DataFrame.from_dict(low_ctr, orient = 'index', columns = ['lowest ctr returns'])
dfhc = pd.DataFrame.from_dict(high_ctr, orient = 'index', columns =['highest ctr returns'])

dfc = pd.concat([dflc,dfhc],axis = 1)

dfc['hedged_returns'] = dfhc['highest ctr returns'] - dflc['lowest ctr returns']

dfc_mean = dfc.mean().to_frame(name = 'Average').T
dfc_std = dfc.std().to_frame(name = 'STD').T

dfc = pd.concat([dfc,dfc_mean,dfc_std])

final_dataframes.append(dfc)

In [ ]:
#Asset Turnover Signal

data['revenue_assets'] = data['revt']/data['at']

grouped = data.groupby(['year'])

low_ra = {}
high_ra = {}

for name, group, in grouped:
    group['ra_percentiles'] = group['revenue_assets'].rank(pct = True)

    lowest_ra = group.loc[group['ra_percentiles'] < 0.1]

    low_ra[name] = lowest_ra['ret'].mean()

    highest_ra = group.loc[group['ra_percentiles'] > 0.9]

    high_ra[name] = highest_ra['ret'].mean()


dflra = pd.DataFrame.from_dict(low_ra, orient = 'index', columns = ['lowest ra returns'])
dfhra = pd.DataFrame.from_dict(high_ra, orient = 'index', columns =['highest ra returns'])

dfar = pd.concat([dflra,dfhra],axis = 1)

dfar['hedged_returns'] = dfhra['highest ra returns'] - dflra['lowest ra returns']

dfar_mean = dfar.mean().to_frame(name = 'Average').T
dfar_std = dfar.std().to_frame(name = 'STD').T

dfar = pd.concat([dfar,dfar_mean,dfar_std])

#print(dfar)

final_dataframes.append(dfar)

In [ ]:
#composite returns PB and Asset Turnover

grouped = data.groupby(['year'])

hcr = {}
lcr = {}

for name, group in grouped:
    group['ra_percentiles'] = group['revenue_assets'].rank(pct = True)
    group['pb_percentiles'] = group['PB'].rank(pct=True)
    group['comp_sig'] = 0.5*(1-group['pb_percentiles'])+0.5*group['ra_percentiles']
    group['comp_sig_pct'] = group['comp_sig'].rank(pct = True)
    
    highest_comp = group.loc[group['comp_sig_pct'] > 0.9]

    hcr[name] = highest_comp['ret'].mean()

    lowest_comp = group.loc[group['comp_sig_pct']<0.1]

    lcr[name] = lowest_comp['ret'].mean()
    
dflcr = pd.DataFrame.from_dict(lcr, orient = 'index', columns = ['lowest comp returns'])
dfhcr = pd.DataFrame.from_dict(hcr, orient = 'index', columns =['highest comp returns'])

dfcr = pd.concat([dflcr,dfhcr],axis = 1)

dfcr['hedged_returns'] = dfhcr['highest comp returns'] - dflcr['lowest comp returns']

dfcr_mean = dfcr.mean().to_frame(name = 'Average').T
dfcr_std = dfcr.std().to_frame(name = 'STD').T

dfcr = pd.concat([dfcr,dfcr_mean,dfcr_std])

final_dataframes.append(dfcr)

In [ ]:
#Write to Excel Sheets
with pd.ExcelWriter('RSM429 Group Analytics Output.xlsx') as writer:
    i = 1
    for df in final_dataframes:
        sheet_name = f'Q{i}'
        df.to_excel(writer, sheet_name)
        i += 1